In [2]:
import pandas as pd

# 각 비트코인 종목 데이터 파일 경로
BTCUSDT_path = 'C:\\education\\ml_dev\\coin\jh_-\\binance\\3m_BTCUSDT.csv'

# CSV 파일을 데이터프레임으로 읽기
BTCUSDT_df = pd.read_csv(BTCUSDT_path)

# 시계열 데이터를 하나의 데이터프레임으로 합치기
# merged_df = pd.concat([BTCUSDT_df])

# 합친 데이터프레임을 CSV 파일로 저장하기
BTCUSDT_df.to_csv('bitcoin_data.csv', index=False)

In [3]:
BTCUSDT_df.head(3)

,open_time,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,symbol
0,2023-10-14 09:00:00,26849.7,26858.1,26843.4,26843.4,261.516,7.022053e+06,2337,106.669,2.864295e+06,BTCUSDT
1,2023-10-14 09:01:00,26843.5,26843.5,26838.6,26843.4,95.056,2.551347e+06,1002,44.023,1.181565e+06,BTCUSDT
2,2023-10-14 09:02:00,26843.4,26854.3,26843.4,26850.0,95.366,2.560510e+06,1155,62.120,1.667846e+06,BTCUSDT


In [4]:
BTCUSDT_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133921 entries, 0 to 133920
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   open_time    133921 non-null  object 
 1   open         133921 non-null  float64
 2   high         133921 non-null  float64
 3   low          133921 non-null  float64
 4   close        133921 non-null  float64
 5   volume       133921 non-null  float64
 6   quote_av     133921 non-null  float64
 7   trades       133921 non-null  int64  
 8   tb_base_av   133921 non-null  float64
 9   tb_quote_av  133921 non-null  float64
 10  symbol       133921 non-null  object 
dtypes: float64(8), int64(1), object(2)
memory usage: 11.2+ MB


In [8]:
BTCUSDT_df = BTCUSDT_df[(BTCUSDT_df==0).any(axis=1)]
print(BTCUSDT_df)

Empty DataFrame
Columns: [open_time, open, high, low, close, volume, quote_av, trades, tb_base_av, tb_quote_av, symbol]
Index: []


---
- 시계열 데이터 시퀀스로 변환

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# 시계열 데이터를 시퀀스로 변환하는 함수
def create_sequences(data, sequence_length, target_step):
    xs = []
    ys = []
    for i in range(len(data) - sequence_length - target_step + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length + target_step - 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# BTCUSDT_df 데이터 프레임에서 'close' 컬럼만 사용하여 시퀀스 데이터 생성
sequence_length = 60  # 1시간 데이터를 입력으로 사용
target_step = 5  # 5분 후를 예측
data_close = BTCUSDT_df['close'].values.reshape(-1, 1)

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
data_close_scaled = scaler.fit_transform(data_close)

# 시퀀스 생성
X, y = create_sequences(data_close_scaled, sequence_length, target_step)

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM 모델에 입력할 수 있도록 데이터 차원 조정
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)